In [ ]:
#回溯时使用所有数据
#与t-1,t-2价格区间比较
#价格区间外0BP
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import networkx as nx
import igraph
import datetime
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import math
import matplotlib.colors as mcolors
import os
import pyecharts
from pyecharts.globals import CurrentConfig, OnlineHostType
CurrentConfig.ONLINE_HOST = "..\\..\\..\\..\\pyecharts-assets-master\\assets\\"
from pyecharts import options as opts
from pyecharts.charts import Graph
plt.rcParams["font.sans-serif"]=["SimHei"]



#alldf = pd.read_excel("C:\\Users\\sys\\Downloads\\190210新.xlsx",encoding="GBK")
alldf = pd.read_csv("C:\\Users\\sys\\Downloads\\111.csv")


In [ ]:

datestr="结算日"
amtstr='券面总额'
fromstr="买入方"
tostr="卖出方"
pricestr="到期收益率(%)"
dlmthdstr="报价成交方式"
odateformat="%Y-%m-%d"
dateformat="%Y/%m/%d"
matchingmthdstr="匿名点击"
bondstr="债券名称"

marketpricethres=0.05
price_ratio_min=0.015
single_price_boundary_min=2
traget_price_ratio_min=0.05
path="daichiresult1"

In [ ]:

datestr="stlmnt_dt"
trddatestr="txn_dt"
amtstr='nmnl_vol'
fromstr="byr_trd_acnt_cn_full_nm"
tostr="slr_trd_acnt_cn_full_nm"
pricestr="yld_to_mrty"
dlmthdstr="trdng_mthd_cd"
odateformat="%Y-%m-%d"
dateformat="%Y/%m/%d"
matchingmthdstr="Matching"
bondstr="bond_cd"

marketpricethres=0
price_ratio_min=-1
single_price_boundary_min=2
traget_price_ratio_min=0
path="version10"

if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
import pandas as pd
import numpy as np

def get_trade_delta(tradedf):
    # 设置价格范围并创建标签
    minprice = int(tradedf[pricestr].min() * 100 - 10)
    maxprice = int(tradedf[pricestr].max() * 100 + 10)
    fanwei = np.arange(minprice, maxprice) * 0.01
    tradedf["label"] = pd.cut(tradedf[pricestr], bins=fanwei, right=False)
    
    # 计算交易持续天数
    unique_dates = tradedf[datestr].unique()
    duration = (unique_dates.max() - unique_dates.min()) / np.timedelta64(1, 'D')
    
    # 初始化新列
    tradedf["day"] = 0
    tradedf["平均收益率"] = 0
    tradedf["delta"] = 0
    tradedf["deltaabs"] = 0
    
    # 遍历每一天的数据
    for i, curdate in enumerate(pd.date_range(start=unique_dates.min(), periods=int(duration) + 1)):
        tradedaydf = tradedf[tradedf[datestr] == curdate]
        a = (tradedaydf[amtstr] * tradedaydf[pricestr]).sum() / (tradedaydf[amtstr].sum() + 1e-6)
        tradedf.loc[tradedaydf.index, "delta"] = tradedaydf[pricestr] - a
        tradedf.loc[tradedaydf.index, "deltaabs"] = np.abs(tradedaydf[pricestr] - a)
        tradedf.loc[tradedaydf.index, "day"] = i
        tradedf.loc[tradedaydf.index, "平均收益率"] = a
    
    # 初始化价格区间
    matchupprice1 = matchdownprice1 = tradedf.loc[tradedf[dlmthdstr] == matchingmthdstr, pricestr].agg(['max', 'min']).values
    matchupprice2, matchdownprice2 = matchupprice1, matchdownprice1
    
    # 遍历交易日期，计算out字段
    for i, curdate in enumerate(pd.date_range(start=unique_dates.min(), periods=int(duration) + 1)):
        tradedaydf = tradedf[tradedf[trddatestr] == curdate]
        if len(tradedaydf) > 1:
            matchupprice0 = tradedaydf.loc[tradedaydf[dlmthdstr] == matchingmthdstr, pricestr].agg(['max', 'min']).values
            matchupprice = max(matchupprice1[0], matchupprice2[0], matchupprice0[0])
            matchdownprice = min(matchdownprice1[1], matchdownprice2[1], matchupprice0[1])
            tradedf.loc[tradedaydf.index, "out"] = tradedf.loc[tradedaydf.index, pricestr].apply(
                lambda x: max(0, x - matchupprice) if x > matchupprice else max(0, matchdownprice - x))
            matchupprice1, matchdownprice1 = matchupprice0
            matchupprice2, matchdownprice2 = matchupprice1, matchdownprice1
    
    # 添加价格区间和价格标记
    tradedf["价格区间"] = tradedf["label"].apply(lambda x: float(x.left))
    tradedf["价格标记"] = tradedf["label"].apply(lambda x: int(x.left * 100))
    
    return tradedf
